# **Hackerearths ML competition Reduce Marketing Waste by pycaret & catboost**

**Note 1:** *Every line was written by me, with methods invented by other people, with PC given by wife*

**Note 2:** *Sorry for my eng*

**Note 3:** *Kurt would have loved python*

**Note 4:** *I started too late, so my first model's predictions were taken only. (It was 80%)
Half an hour was not enough for me to drop useless features and train my model again! I was upset.*

In [ ]:
# libraries I'll need
# for some reason pycaret make unfriendly to standard matplotlib
!pip install matplotlib==3.1.1
!pip install pycaret
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme(palette='pastel')
import warnings
warnings.filterwarnings('ignore')
from pycaret.regression import *
from catboost import CatBoostRegressor
from catboost import cv
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# **Step 1.** Downloading and getting familiarize with data

In [ ]:
#downloading
train_data = pd.read_csv('../input/hackerearths-reduce-marketing-waste/train.csv')
test_data = pd.read_csv('../input/hackerearths-reduce-marketing-waste/test.csv')
# using the best weirds-searching method (imho)
train_data.info()

In [ ]:
#yet again
test_data.info()

In [ ]:
# viewing a couple simple things
# train_data.head()
test_data.tail()

In [ ]:
# some lines below to know target column
for i in train_data.columns:
    if i not in test_data.columns:
        print('-------------------------------------\n| Need to find:', i, '|\n-------------------------------------')

In [ ]:
# looks like regression task
train_data.Success_probability.head()

In [ ]:
# looking at target's distribution
sns.histplot(train_data.Success_probability, kde=True, color='r')

In [ ]:
good_data = train_data[(train_data.Success_probability > 40) & (train_data.\
Success_probability < 101)]
# checking size
len(good_data)

In [ ]:
# nice
sns.histplot(good_data.Success_probability, element='step', kde=True, color='r')

In [ ]:
# by the way, let's find all features we'll fill
print('Good_data:\n')
for i in good_data.drop('Success_probability', axis=1).columns:
    x = good_data[i].isna().sum()
    if x > 0:
        print('good_data_' + i + ' - ' + str(x))
print('---------------------------------\n\n', 'Test_data:\n')
for i in test_data.columns:
    x = test_data[i].isna().sum()
    if x > 0:
        print('test_data_' + i + ' - ' + str(x))

# **Step 2.** EDA

In [ ]:
# looking at our data again to find hidden features
good_data.head()

In [ ]:
# crating some useful lists for keeping features:
to_drop = []
to_get_dummies = []

In [ ]:
# notice that we have six different types of companies
companies = ['ltd', 'llc', 'inc', 'group', 'plc', 'sons']

def company(s):
    for i in range(6):
        if s.lower().endswith(companies[i]):
            return companies[i]
    return 'other'

good_data.Lead_name = good_data.Lead_name.apply(company)
test_data.Lead_name = test_data.Lead_name.apply(company)

to_get_dummies.append('Lead_name')

In [ ]:
# Contact_no feature is useless
to_drop.append('Contact_no')

In [ ]:
# filling missing Industry values
good_data.Industry.loc[good_data.Industry.isna()] = 'Other'
test_data.Industry.loc[test_data.Industry.isna()] = 'Other'

In [ ]:
# what do these banks need,they'd better give me a job
df = good_data.Industry.value_counts().to_frame()\
.rename(columns={'Industry' : 'Number_of_deals'})
pd.concat([df.head(), df.tail()])

In [ ]:
# we have many different ways to solve "Industry" problem, so there are:
# 1) making something like bank/not_bank feature, 
# 2) putting low-used values (Number_of_deals < X, e.g. X = 20) in "Other" value
# 3) creating new groups (for instance IT, that would include Web, Web development, etc.)
# 4) sending to get_dummies without any changes
# 5) dropping Industry feature
# but let's go by the 2nd way:

df = df.head(40)

def foo(s):
    return s if s in df.index else 'Other'
        
good_data.Industry = good_data.Industry.apply(foo)
test_data.Industry = test_data.Industry.apply(foo)

to_get_dummies.append('Industry')

In [ ]:
# looking at "Deal_value" and "Weighted_amount" features
good_data[['Deal_value', 'Weighted_amount']].head()

In [ ]:
# line below gives good opportunity for data recovery, but it seems no good...
# good_data[good_data.Deal_value.isnull()][['Deal_value', 'Weighted_amount']]
good_data[good_data.Weighted_amount.isnull()][['Deal_value', 'Weighted_amount']].head()

In [ ]:
# making temporary dataframes
good_wa = good_data[(good_data.Weighted_amount\
.isnull() == False) & (good_data.Deal_value.isnull() == False)]

test_wa = test_data[(test_data.Weighted_amount\
.isnull() == False) & (test_data.Deal_value.isnull() == False)]

good_wa.head()

In [ ]:
# converting string to numbers
def to_number(s):
    return float(s[:-1])

good_wa.Deal_value = good_wa.Deal_value.apply(to_number)
good_wa.Weighted_amount = good_wa.Weighted_amount.apply(to_number)

test_wa.Deal_value = test_wa.Deal_value.apply(to_number)
test_wa.Weighted_amount = test_wa.Weighted_amount.apply(to_number)

In [ ]:
# we know that Weighted_amount ten times more "missed" than Deal_value but...
sns.jointplot(data=good_wa, x='Deal_value', y='Weighted_amount', color='r')
# its two almost mutually (linearly) dependent features!

In [ ]:
# some cells below used for both dataframes at once (it would work better with creating some funcs)
# let's find this dependence coefficient
this_coef = (good_wa.Deal_value / good_wa.Weighted_amount).mean()
this_t_coef = (test_wa.Deal_value / test_wa.Weighted_amount).mean()
this_coef

In [ ]:
# Deal_value/Success_probability graph
sns.jointplot(data=good_wa, x='Deal_value', y='Success_probability', color='r')

In [ ]:
# at first glance "Weighted_amount" feature gives a slightly better correlation, so...
sns.jointplot(data=good_wa, x='Weighted_amount', y='Success_probability', color='r')
# ...let's check

In [ ]:
# cotcha
sns.heatmap(good_wa[['Deal_value', 'Weighted_amount', 'Success_probability']]\
            .corr(), annot=True, cmap="YlGnBu")

In [ ]:
# $$$
wa_list = good_data[good_data.Weighted_amount.isnull() == False].Weighted_amount.to_list()
wa_test = test_data[test_data.Weighted_amount.isnull() == False].Weighted_amount.to_list()
wa_list[:5]
# making lists with available values

In [ ]:
# before dropping "Deal value", we'll make some magics to fill some Weighted_amount's NaNs

good_data.Weighted_amount.loc[good_data[(good_data.Deal_value.isnull() == False) & (good_data.Weighted_amount\
.isnull())].index] = good_data[(good_data.Deal_value.isnull() == False) & (good_data.Weighted_amount\
.isnull())].Deal_value.apply(to_number) / this_coef

test_data.Weighted_amount.loc[test_data[(test_data.Deal_value.isnull() == False) & (test_data.Weighted_amount\
.isnull())].index] = test_data[(test_data.Deal_value.isnull() == False) & (test_data.Weighted_amount\
.isnull())].Deal_value.apply(to_number) / this_t_coef

In [ ]:
# deleting $
def fill_vals(s):
    return float(s[:-1]) if s in wa_list else s

good_data.Weighted_amount = good_data.Weighted_amount.apply(fill_vals)

def fill_vals_2(s):
    return float(s[:-1]) if s in wa_test else s

test_data.Weighted_amount = test_data.Weighted_amount.apply(fill_vals_2)

In [ ]:
# we need to know mean of "Weighted_amount" feature
mean_wa = good_data[good_data.Weighted_amount.isnull() == False].Weighted_amount.mean()
mean_t_wa = test_data[test_data.Weighted_amount.isnull() == False].Weighted_amount.mean()

In [ ]:
# filling all missed values, by the way, appending useless feature to "to_drop" list
def round_(s):
    return round(s, 2)

good_data.Weighted_amount = good_data.Weighted_amount.fillna(mean_wa).apply(round_)
test_data.Weighted_amount = test_data.Weighted_amount.fillna(mean_wa).apply(round_)

to_drop.append('Deal_value')

In [ ]:
# it's time to explore the time
def times(data):
    data['Year_of_creation'] = pd.to_datetime(data.Date_of_creation).dt.year.apply(str)
    data['Month_of_creation'] = pd.to_datetime(data.Date_of_creation).dt.month.apply(str)
    
times(good_data)
times(test_data)

print('Train_years:', list(good_data.Year_of_creation.unique()))
print('Test_years:', list(test_data.Year_of_creation.unique()))

to_get_dummies.append('Year_of_creation')
to_get_dummies.append('Month_of_creation')
to_drop.append('Date_of_creation')

In [ ]:
# easy to convert because "Pitch" feature has two values only
def pitch_to(df):
    
    def to_zero(s):
        return 0 if s == 'Product_1' else 1

    df.Pitch = df.Pitch.apply(to_zero)

pitch_to(good_data)
pitch_to(test_data)

In [ ]:
# sending to get_dummies
print('Lead_revenue: ', good_data.Lead_revenue.unique())
print('Fund_category: ',good_data.Fund_category.unique())

to_get_dummies.append('Lead_revenue')
to_get_dummies.append('Fund_category')

In [ ]:
print('TRAIN:\nContries are', ' and '.join(good_data.Geography\
.dropna().unique()), 'with', good_data.Location.nunique(), 'states')
print('------------------------------------------')
print('TEST:\nContries are', ' and '.join(good_data.Geography\
.dropna().unique()), 'with', test_data.Location.nunique(), 'states')

In [ ]:
# there are some cases with missed locations,
# however, geography features are available at the same time
good_data[good_data.Location.isna()][['Geography', 'Location']]

In [ ]:
# check for missed countries, so Randy Ramos will send to Texas later
test_data[(test_data.Location.isna()) & (test_data.Geography.isna())][['Geography', 'Location', 'POC_name']]

In [ ]:
def state_USA(s):
    
    return s[-2:].isupper()

def state_USA2(s):
    
    return s[-2:]

print('TRAIN:')

good_data.Location = good_data.Location.fillna('Other')
states = good_data[good_data.Location.apply(state_USA)].Location.apply(state_USA2)

print(states.value_counts().to_frame().T)
states_list = list(states)
states_set = set(states)
print('-----------------------------\nNumber of states in data: ' + str(states.value_counts().shape[0]))

USA_states_taken_from_WWW = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "OR", "DE", "FL",
                             "GA", "HI" , "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME",
                             "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH",
                             "NJ", "NM", "NY", "NC", "ND", "OH", "OK", 'PA', 'RI', 'SC',
                             'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI',"WY"]

print('-----------------------------\nNumber of states: ' + str(len(USA_states_taken_from_WWW)))

for i in USA_states_taken_from_WWW:
    if i not in states_set:
        print('-----------------------------\nMissed state: '+ i)
# and now we are choosing again, one of to ways:
# 1) use random.choice(states_list)
# 2) use RI instead of NaN-values

In [ ]:
# the 2nd way 
good_data.Location.loc[good_data[good_data.Location.apply(state_USA)].index] = states
good_data.Location.loc[good_data[good_data.Location == 'Other'].index] = 'RI'
good_data[good_data.Location == 'RI']

In [ ]:
# checking changes
good_data[good_data.Geography.isna()][['Geography', 'Location']]

In [ ]:
# Hoped for easter eggs, I even had been searching on U.S. Bureau's of
# Economic Analysis site for a few hours, but...
set_India_states = set(good_data[good_data.Location.apply(state_USA) == False].Location)
strange_list, x = [], []

for i in set_India_states:
    if len(i.strip().split(' ')) > 2:
        strange_list.append(i)
        print(i)
        
for i in strange_list:
    set_India_states.remove(i)
    x.append(good_data.loc[good_data[good_data.Location == i].index])

dat = pd.concat(x, axis=0)
dat
# neither site, nor data, nor Answer to the Ultimate Question of Life,
# the Universe, and Everything, do not show me any hint

In [ ]:
# good
def filling_missed_countries(s):
    
    return 'India' if s in set_India_states else 'USA'

good_data.Geography = good_data.Location.apply(filling_missed_countries)
print(good_data.Geography.unique())

In [ ]:
# repeating with test
test_data[test_data.Location.isna()][['Geography', 'Location']]

In [ ]:
# repeating with test
def state_USA(s):
    
    return s[-2:].isupper()

def state_USA2(s):
    
    return s[-2:]

print('TEST:')

test_data.Location = test_data.Location.fillna('Other')
states = test_data[test_data.Location.apply(state_USA)].Location.apply(state_USA2)

print('Group by states:\n\n', states.value_counts().to_frame().T)
states_list = list(states)
states_set = set(states)
print('\n-----------------------------\nNumber of states in data: ' + str(states.value_counts().shape[0]))

USA_states_taken_from_WWW = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "OR", "DE", "FL",
                             "GA", "HI" , "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME",
                             "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH",
                             "NJ", "NM", "NY", "NC", "ND", "OH", "OK", 'PA', 'RI', 'SC',
                             'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI',"WY"]

print('---------------------\nNumber of states: ' + str(len(USA_states_taken_from_WWW)))

for i in USA_states_taken_from_WWW:
    if i not in states_set:
        print('-----------------\nMissed state: '+ i)

In [ ]:
# Randy Ramos at home, by the way, Texas was part of Mexico until 1836
test_data.Location.loc[test_data[test_data.Location.apply(state_USA)].index] = states
test_data.Location.loc[test_data[(test_data.\
Location == 'Other') & (test_data.Geography.isna())].index] = 'TX'
test_data.Location.loc[test_data[test_data.Location == 'Other'].index] = 'RI'
test_data[test_data.Location == 'RI']

In [ ]:
set_India_states = set(test_data[test_data.Location.apply(state_USA) == False].Location)
strange_list, x = [], []

for i in set_India_states:
    if len(i.strip().split(' ')) > 2:
        strange_list.append(i)
        print(i)
        

for i in strange_list:
    set_India_states.remove(i)
    x.append(test_data.loc[test_data[test_data.Location == i].index])

dat = pd.concat(x)
dat

In [ ]:
def filling_missed_countries(s):
    
    return 'India' if s in set_India_states else 'USA'

test_data.Geography = test_data.Location.apply(filling_missed_countries)
print(test_data.Geography.unique())
# phewww... next time, we'll make some function without such complexity

In [ ]:
# sorry to guys from the USA, but first seven places received by guys from India
def binomial_geo(s):
    
    return 1 if s == 'India' else 0

for data in [good_data, test_data]:
    data['Country'] = data.Geography.apply(binomial_geo)

to_drop.append('Geography')

In [ ]:
df_1 = good_data.Location.value_counts().to_frame()\
.rename(columns={'Location' : 'Number_of_deals'})
df_1.head()

In [ ]:
df_2 = test_data.Location.value_counts().to_frame()\
.rename(columns={'Location' : 'Number_of_deals'})
df_2.tail()

In [ ]:
# should I drop Location feature? or...
# should I drop Country feature? or...
# let it be
# but we need to find...
to_other_1, to_other_2 = [], []
for i in df_1.index:
    if i not in df_2.index:
        to_other_1.append(i)

for i in df_2.index:
    if i not in df_1.index:
        to_other_2.append(i)      
print('Missed in test: ' + ', '.join(to_other_1) + '\nMissed in train:' + ', '.join(to_other_2))
# ...values to drop

In [ ]:
def to_oth(s):
    return s if s in df_1.head(50).index else 'Other'

test_data.Location = test_data.Location.apply(to_oth)
good_data.Location = good_data.Location.apply(to_oth)
good_data.Location.head()

In [ ]:
to_get_dummies.append('Location')

In [ ]:
# getting some useful (probably) features 
def separate(s):
    return set(map(lambda x: x.strip(), s.split('/')))

good_data.Designation = good_data.Designation.apply(separate)
test_data.Designation = test_data.Designation.apply(separate)


good_data.Designation.head(3)

In [ ]:
# still working
set_designation = set()

for i in good_data.Designation:
    set_designation.update(i)

for i in test_data.Designation:
    set_designation.update(i)

# we make something like one hot engoding there    
for i in set_designation:
    good_data[i] = pd.Series([1 if i in j else 0 for j in good_data\
                              .Designation], index=good_data.index)
    test_data[i] = pd.Series([1 if i in j else 0 for j in test_data\
                              .Designation], index=test_data.index)
to_drop.append('Designation')

In [ ]:
# sent to get_dummies
to_get_dummies.append('Lead_source')
good_data.Lead_source.unique()

In [ ]:
# lvl_of_meeting_feature looks good
to_get_dummies.append('Level_of_meeting')
sns.boxplot(data=good_data, x='Level_of_meeting', y='Success_probability')
# and we see it's importance

In [ ]:
# preparing for get_dummies
good_data.Last_lead_update = good_data.Last_lead_update.fillna('Other_1')
good_data.Last_lead_update.loc[good_data[good_data.Last_lead_update == '?'].index] = 'Other_2'

test_data.Last_lead_update = test_data.Last_lead_update.fillna('Other_1')
test_data.Last_lead_update.loc[test_data[test_data.Last_lead_update == '?'].index] = 'Other_2'

print(good_data.Last_lead_update.unique())
print(test_data.Last_lead_update.unique())
to_get_dummies.append('Last_lead_update')

In [ ]:
# we will assume that number of given cases are almost equal
df1 = good_data.Last_lead_update.value_counts().to_frame()\
.rename(columns={'Last_lead_update' : 'TRAIN'})
df2 = test_data.Last_lead_update.value_counts().to_frame()\
.rename(columns={'Last_lead_update' : 'TEST'})
df1.join(df2).T

In [ ]:
def llu(s):
    k = 0
    for i in good_data.Last_lead_update.unique():
        k += 1
        if i == s:
            return k

# The most strange thing is...
sns.boxplot(data=good_data, x=good_data.Last_lead_update.apply(llu), y='Success_probability')
# ...any position (e. g "Did not hear back after Level 1") is as good as others

In [ ]:
# good to us is Internal_POC's identity
ipsum = 0
ip = good_data.Internal_POC.unique()
print(len(ip), ip)
for i in ip:
    if i in test_data.Internal_POC.unique():
        ipsum += 1
print(ipsum)
to_get_dummies.append('Internal_POC')

In [ ]:
# gotten lucky again
# print(good_data.Resource.unique())

good_data.Resource = good_data.Resource.fillna('Other')
test_data.Resource = test_data.Resource.fillna('Other')

print(good_data.Resource.unique())
print(test_data.Resource.unique())
to_get_dummies.append('Resource')

In [ ]:
# well...
for data in [good_data, test_data]:
    print(data.Internal_rating.unique())
# ...not good... 

In [ ]:
# ...cause we have a large dependence between
# Success_probability & Internal_rating
sns.boxplot(data=good_data, x='Internal_rating', y='Success_probability')

In [ ]:
def visualize(df):
    print(data.Internal_rating.value_counts().to_frame().reset_index()\
    .rename(columns={'Internal_rating' : 'Amount',
                 'index' : 'Internal_rating'}))

visualize(good_data)

In [ ]:
visualize(test_data)

In [ ]:
print(test_data[test_data.Internal_rating == -1]\
.shape[0] + test_data[test_data.Internal_rating == 82.34].shape[0], 'lines with missed Internal_rating :(')

In [ ]:
# there is a good one

graph_2 = sns.boxplot(data=good_data, x='Lead_revenue', y='Success_probability')

In [ ]:
# useless feature
to_drop.append('Lead_POC_email')
good_data.Lead_POC_email.head()

In [ ]:
# dropping Hiring_candidate_role because
for i in test_data.Hiring_candidate_role:
    if i in good_data.Hiring_candidate_role:
        print(i)
to_drop.append('Hiring_candidate_role')

In [ ]:
# one more useless
to_drop.append('POC_name')

In [ ]:
# there are no feature with missed values, except for the "POC_name" feature to be deleted
good_data.info()

# **Step 3.** Preparing for our model

In [ ]:
# check our to_get_dummies list, there are 8 features
to_get_dummies

In [ ]:
# check our to_drop list
to_drop

In [ ]:
good_data = pd.get_dummies(good_data, columns=to_get_dummies)
test_data = pd.get_dummies(test_data, columns=to_get_dummies)

In [ ]:
# looking for few columns again, to drop 1 degree of freedom per one old feature
# it makes our model performance better
good_data.columns.to_list()[20:40]

In [ ]:
drop_too = ['Industry_Biotech/Healthcare',
            'Lead_revenue_100 - 500 Million',
            'Level_of_meeting_Level 1',
            'Year_of_creation_2019',
            'Lead_name_inc',
            'Resource_Other',
            'Location_PA',
            'Internal_POC_Young,Valerie K',
            'Last_lead_update_more than a month',
            'Lead_source_Others',
            'Fund_category_Category 4',
            'Month_of_creation_12'
           ]

In [ ]:
good_data = good_data.drop(to_drop + drop_too, axis=1)
test_data = test_data.drop(to_drop + drop_too, axis=1)

In [ ]:
# preparing test data for using by two models, they are: 
# with "Internal_rating" feature
test_with_ir = test_data[(test_data.Internal_rating > 0) & (test_data\
                                                        .Internal_rating < 10)]

test_with_ir.Internal_rating = test_with_ir.Internal_rating.apply(int)

test_with_ir = pd.get_dummies(test_with_ir, columns=['Internal_rating'])

test_with_ir = test_with_ir.drop('Internal_rating_5', axis=1)

# without "Internal_rating" feature"
test_without = test_data[(test_data.Internal_rating < 0) | (test_data\
               .Internal_rating > 10)].drop('Internal_rating', axis=1)

In [ ]:
# same
good_without = good_data.drop('Internal_rating', axis=1)

good_with_ir = pd.get_dummies(good_data, columns=['Internal_rating']).drop('Internal_rating_5', axis=1)

In [ ]:
cat_feats_1 = good_with_ir.drop(['Success_probability', 'Weighted_amount', 'Deal_title'], axis=1).columns.to_list()
cat_feats_2 = good_without.drop(['Success_probability', 'Weighted_amount', 'Deal_title'], axis=1).columns.to_list()

# **Step 4.** Building first model & working with it by using pycaret & catboost

**Note 5:** *Don't panic and don't devide your data by* **X** *and* **Y**, *pycaret can handle it himself, put name of your y-vector to "target="*

In [ ]:
# this model is giving to us opportunity to compare different models, by different metrics, but
# we need MSE
model_1 = setup(data = good_with_ir.drop('Deal_title', axis=1),
             numeric_features=['Weighted_amount'],
             target = 'Success_probability',
             categorical_features=cat_feats_1,
             ignore_low_variance=True,
             fold = 4,
             silent = True,
             normalize=True)

In [ ]:
# it prefers catboost
best_model = compare_models(sort='MSE')

In [ ]:
# its time to prepare our first model for catboost
y_1 = good_with_ir.Success_probability
X_1 = good_with_ir.drop(['Deal_title', 'Success_probability'], axis=1)

In [ ]:
# creation of simple model to investigate features better
# catboost haven't MSE metric included
# I already competed, so I'm not going to add MSE to this model, but it's possible
cat = CatBoostRegressor(eval_metric='RMSE',
                        n_estimators=2000,
                       depth=5)

In [ ]:
# we, actually, will not draw any graph, we only need to know which features should to be dropped
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.2)

In [ ]:
cat.fit(X_train_1, y_train_1,
        verbose=False,
        eval_set=(X_test_1, y_test_1),
        cat_features=cat_feats_1,
        use_best_model=True)

In [ ]:
# checking mean squared error
print('MSE:', np.sqrt(metrics.mean_squared_error(y_test_1, cat.predict(X_test_1))))

In [ ]:
table_1 = pd.DataFrame(cat.get_feature_importance(), X_train_1.columns).reset_index()\
.rename(columns={'index' : 'Feature',
                0 : 'Importance'})

table_1.sort_values(by='Importance', ascending=False).head(15)

In [ ]:
good_features_1 = table_1[table_1.Importance > 1.4].Feature.to_list()
good_features_1

In [ ]:
# splitting by categorical and numeric
good_cat = good_features_1[0]

#  better train data
X_1 = X_1[good_features_1]
#  better test data
X_P_1 = test_with_ir[good_features_1]

good_features_1 = good_features_1[1:]

In [ ]:
# competition was stopped there, so I rewrite a little, test data was completed by this moment too
# ohhh... he apologizes again
cat_0 = CatBoostRegressor(eval_metric='RMSE',
                          n_estimators=1000,
                          depth=5,
                          cat_features=good_features_1)

In [ ]:
# setting
params_grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 10],
        'l2_leaf_reg': [3, 10, 25]}

In [ ]:
# searching best parameters
grid_search_result = cat_0.grid_search(params_grid, 
                                       X=X_1, 
                                       y=y_1,
                                       cv=5,
                                       search_by_train_test_split=True,
                                       verbose=0)

In [ ]:
# so they are
param_dict = grid_search_result['params']
param_dict

In [ ]:
# harder better faster stronger
cat_1 = CatBoostRegressor(eval_metric='RMSE',
                          n_estimators=1000,
                          **param_dict)

In [ ]:
# fitting
cat_1.fit(X_1,
          y_1,
          cat_features=good_features_1,
          verbose=0)

In [ ]:
# results of with-Internal_rating part
the1st = cat_1.predict(X_P_1, verbose=0)

In [ ]:
# deleting Internal_rating
good_features_1 = good_features_1[:-4]

In [ ]:
# using pycaret again and then comparing
# model_2 = setup(data = good_without,
#              numeric_features=['Weighted_amount'],
#              target = 'Success_probability',
#              categorical_features=cat_feats_2,
#              ignore_low_variance=True,
#              fold = 4,
#              silent = True,
#              normalize=True)

In [ ]:
# it says catboost again
# compare_models()

In [ ]:
good_features_1

In [ ]:
# so repeating

cat_2 = CatBoostRegressor(eval_metric='RMSE',
                          n_estimators=1000,
                          **param_dict)

In [ ]:
good_features_1

In [ ]:
good_features_1.append(good_cat)

X_2 = X_1[good_features_1]
X_P_2 = test_without[good_features_1]

good_features_1 = good_features_1[:-1]

In [ ]:
cat_2.fit(X_2,
          y_1,
          cat_features=good_features_1,
          verbose=0)


the2nd = cat_2.predict(X_P_2)

In [ ]:
test_with_ir['Success_probability'] = the1st
test_without['Success_probability'] = the2nd

In [ ]:
test_with_ir = test_with_ir[['Deal_title', 'Success_probability']]
test_without = test_without[['Deal_title', 'Success_probability']]        

In [ ]:
ans = pd.concat([test_with_ir, test_without]).sort_index()

In [ ]:
# now let me sleep a little
ans.to_csv('sample_submission.csv')